In [ ]:
import logging
import os

from cods.od.data import MSCOCODataset
from cods.od.models import DETRModel
from cods.od.cp import ODConformalizer
from itertools import product
import pickle

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = (
    "0"  # chose the GPU. If only one, then "0"
)

logging.getLogger().setLevel(logging.DEBUG)

# set [COCO_PATH] to the directory to your local copy of the COCO dataset
COCO_PATH = "/datasets/shared_datasets/coco/"

data = MSCOCODataset(root=COCO_PATH, split="val")

calibration_ratio = (
    0.5  # set 0.5 to use 50% for calibration and 50% for testing
)

use_smaller_subset = True  # TODO: Temp

if use_smaller_subset:
    data_cal, data_val = data.split_dataset(
        calibration_ratio, shuffle=False, n_calib_test=800
    )
else:
    data_cal, data_val = data.split_dataset(calibration_ratio, shuffle=False)

# model and weights are downloaded from https://github.com/facebookresearch/detr
model = DETRModel(model_name="detr_resnet50", pretrained=True, device="cpu")
# model = YOLOModel(model_name="yolov8x.pt", pretrained=True)


print(f"{len(data) = }")
print(f"{len(data_cal) = }")
print(f"{len(data_val) = }")

preds_cal = model.build_predictions(
    data_cal,
    dataset_name="mscoco",
    split_name="cal",
    batch_size=12,
    collate_fn=data._collate_fn,  # TODO: make this a default for COCO
    shuffle=False,
    force_recompute=False,  # False,
    deletion_method="nms",
)
preds_val = model.build_predictions(
    data_val,
    dataset_name="mscoco",
    split_name="test",
    batch_size=12,
    collate_fn=data._collate_fn,
    shuffle=False,
    force_recompute=False,  # False,
    deletion_method="nms",
)

results = {}

alphas = [[0.1, 0.1, 0.1]]
matching_functions = ["mix"]#["giou", "hausdorff"]
confidence_methods = ["box_count_threshold"]
localization_methods = ["pixelwise"]
classification_prediction_sets = ["lac", "aps"]
localization_prediction_sets = ["additive"]

configs = []
for alpha in alphas:
    for matching_function in matching_functions:
        for confidence_method in confidence_methods:
            for localization_method in localization_methods:
                for classification_prediction_set in classification_prediction_sets:
                    for localization_prediction_set in localization_prediction_sets:
                        configs.append({
                            "alpha": alpha, 
                            "matching_function": matching_function,
                            "confidence_method": confidence_method,
                            "localization_method": localization_method,
                            "classification_prediction_set": classification_prediction_set,
                            "localization_prediction_set": localization_prediction_set,
                        })
for config in configs:
    try:
        conf = ODConformalizer(
            guarantee_level="image",
            matching_function=config["matching_function"],
            multiple_testing_correction=None,
            confidence_method=config["confidence_method"],
            localization_method=config["localization_method"],
            localization_prediction_set=config["localization_prediction_set"],
            classification_method="binary",
            classification_prediction_set=config["classification_prediction_set"],
            backend="auto",
            optimizer="binary_search",
        )
        # conf = ODConformalizer(
        #     guarantee_level="image",
        #     matching_function="hausdorff",  # "giou",
        #     multiple_testing_correction=None,
        #     confidence_method="box_count_threshold",  # "nb_boxes",
        #     # confidence_threshold=0.5,
        #     localization_method="pixelwise",  # "pixelwise",
        #     localization_prediction_set="additive",  # "multiplicative",
        #     classification_method="binary",
        #     classification_prediction_set="lac",
        #     backend="auto",
        #     optimizer="binary_search",
        #     #device="gpu",
        # )
            # TODO(leo): we can replace this by anything, doesn't even need a guarantee (confidence)
        parameters = conf.calibrate(
            preds_cal,
            alpha_confidence=config["alpha"][0],
            alpha_localization=config["alpha"][1],
            alpha_classification=config["alpha"][2],
        )
        
        conformal_preds = conf.conformalize(preds_val, parameters=parameters)
        
            # TODO: Rewrite it so we only compute the confidence loss and not the max of three. Main loss of condiecne shoudl be just itself but in calibration use the proxy maximum loss with the others
        results_val = conf.evaluate(
            preds_val,
            parameters=parameters,
            conformalized_predictions=conformal_preds,
            include_confidence_in_global=False,
        )
        
        config_str = f"alpha-{config['alpha']}-{config['matching_function']}_{config['confidence_method']}_{config['localization_method']}_{config['classification_prediction_set']}_{config['localization_prediction_set']}"
        
        results[config_str] = results_val
        
        print(f"Results for config {config_str}:")
        print(f"  {results_val}")
    except Exception as e:
        print(f"Error with config {config}: {e}")
        continue

[2025-04-09 01:27:44:INFO:models.py:models:30 - __init__ ] Model detr_resnet50 initialized
Using cache found in /home/leo.andeol/.cache/torch/hub/facebookresearch_detr_main
/home/leo.andeol/envs/cods_13/cods/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/leo.andeol/envs/cods_13/cods/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[2025-04-09 01:27:48:INFO:models.py:models:76 - _load_preds_if_exists ] Loading predictions from ./saved_predictions/2c92d1aaa0cc2db665dc992cc2c

len(data) = 5000
len(data_cal) = 400
len(data_val) = 400
Predictions already exist, loading them...


[2025-04-09 01:27:48:WARNING:cp.py:cp:1409 - __init__ ] No multiple_testing_correction provided, assuming no correction is needed. The explicit list of alphas is expected for calibration.
[2025-04-09 01:27:48:INFO:cp.py:cp:188 - __init__ ] Defaulting to CRC backend
[2025-04-09 01:27:48:INFO:cp.py:cp:1013 - __init__ ] Defaulting to CRC backend
[2025-04-09 01:27:48:INFO:cp.py:cp:1637 - calibrate ] Calibrating Confidence Conformalizer
[2025-04-09 01:27:48:DEBUG:cp.py:cp:823 - calibrate ] Optimizing for lambda_plus


Predictions already exist, loading them...


  0%|          | 0/400 [00:00<?, ?it/s]/home/leo.andeol/envs/cods_13/cods/cods/od/utils.py:403: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  Qst = torch.FloatTensor([Qs]).to(device)
100%|██████████| 400/400 [00:00<00:00, 1090.68it/s]
[2025-04-09 01:27:54:DEBUG:optim.py:optim:191 - optimize ] Risk after 1st epoch is 0.0024937656708061695 < 0.1


First risk: 0.0024937656708061695


0.9950659275054932. Corrected Risk = 0.0075:  28%|██▊       | 4357/15831 [01:27<04:46, 40.01it/s]